# 11. Appendix
## 11. 1. Computer Code 

In [2]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np
import datetime
import time
import cProfile
import random
from numba import jit, njit, objmode
import lovasz # local file lovasz.py
import math

In [3]:
def verify_k_color(graph, feas_coloring):
    truth = []
    for i, j in graph.edges:
        truth.append(feas_coloring[i] != feas_coloring[j])
    return np.all(np.array(truth))

In [4]:
# Run DSATUR heuristic and find a feasible solution (upper bound)
def get_ub_DSATUR(g, num_ind_sets):
    graph = g.copy(as_view=False)
    best_size = 0
    start = time.time()
    for i in range(num_ind_sets):
        if time.time() - start > 40:
            break
        ind = nx.maximal_independent_set(graph)
        if len(ind) > best_size:
            best_size = len(ind)
            best_ind_set = ind
    print(f"found best ind set of size {best_size}")
    graph.remove_nodes_from(best_ind_set)
        
    print("Running DSATUR heuristic")
    feas_coloring = nx.algorithms.greedy_color(graph, strategy='DSATUR')
    last_color = 0
    for key in feas_coloring:
        last_color = max(last_color, feas_coloring[key])
    
    for v in best_ind_set:
        feas_coloring[v] = last_color + 1 # new color for the clique
    last_color += 1                       # update last color
    
    color_count = last_color + 1 # color number starts from 0 in the algorithm # one more +1 as we are removing clique
    return color_count, feas_coloring

def get_lb_clique(graph, timelimit):
    print("Trying to find a good clique.")
    print(f"Time Limit {timelimit}")
    j = 0
    best_k = 0
    best_cli = {}
    maximal_cliques = nx.find_cliques(graph)
    clq_start = time.time()
    for i in maximal_cliques:
        j += 1
        if best_k < len(i):
            best_k = len(i)
            max_cliq = i
        if j == 1000000 or time.time() - clq_start > timelimit:
            print("found clique of size {best_k}")
            break
    return max_cliq, best_k

In [5]:
#path = r"""F:\Research\Problem sets\Coloring\myciel4.col.txt"""
# path = r"""F:\Research\Problem sets\Coloring\Instances\DSJR500.5.col"""
# path = r"""F:\Research\Problem sets\Max Clique\DIMACS_all_ascii\c-fat500-2.clq""" # needs investigation
#path = r"""E:\Downloads\all-instaces.tar\1-FullIns_5.col"""

p_start_t = time.time()

INSTANCES_PATH = r"""F:\Research\Problem sets\Coloring\CP\Instances"""
OUT_LP_PATH = r"""F:\Research\Problem sets\Coloring\CP\OutputLP"""
SOLVER_LOG_PATH = r"""F:\Research\Problem sets\Coloring\CP\SolverLog"""

instance_name =  "DSJC1000.1" #str(sys.argv[1])

filename = instance_name + r""".col"""
# MAX_TIME_LIMIT_SECS = int(sys.argv[2]) #10
path = INSTANCES_PATH + '\\' + filename
print(instance_name)

print("opponent's play")
go_easy = True
variable_fixing = False
add_user_cuts = False
PRE_PROCESS = False
help_with_bounds = False
isBIP = True
use_lovasz_theta = False
mip_warm_start = False

# Uncomment this section for Method 1
'''
print('Our method')
go_easy = True
variable_fixing = True
add_user_cuts = False
PRE_PROCESS = False
help_with_bounds = True
isBIP = True
use_lovasz_theta = True
mip_warm_start = True
'''

#assert(PRE_PROCESS == help_with_bounds)

f = open(path, "r")
input = f.read()
f.close()
#print(input)
words = input.split()
for idx, word in enumerate(words):
    if word == 'p':
        break


parse1 = [int(i) for i in words[idx+2:] if i!= 'e']
    
num_v = parse1[0]
num_e = parse1[1]
parse1 = parse1[2:]

parse2 = []
for i in range(len(parse1)//2):
    parse2.append((parse1[2*i], parse1[2*i + 1]))


DSJC1000.1
opponent's play


In [6]:
graph = nx.Graph()
graph.add_nodes_from(range(1,num_v+1))
graph.add_edges_from(parse2)
print(f"Number of nodes: {len(graph.nodes)}")
print(f"Number of edges: {len(graph.edges)}")

Number of nodes: 1000
Number of edges: 49629


In [7]:
if instance_name == "DSJC250.9":
    l_theta = 55.15
if instance_name == "DSJC500.5":
    l_theta = 20.542
if instance_name == "DSJR500.1c":
    l_theta = 83.74
if instance_name == "DSJC1000.1":
    l_theta = 8.307
if instance_name == "DSJC1000.5":
    l_theta = 31.89    

In [8]:
l_theta = 0
l_theta_start = 0
if help_with_bounds == True and use_lovasz_theta == True:
    l_theta_start = time.time()
    cg = nx.complement(graph)
    l_theta = lovasz.theta(cg, False)
print(time.time() - l_theta_start)

1588996730.3049972


In [152]:
print(f"Lovasz-theta = {l_theta}")

Lovasz-theta = 8.307


In [153]:
# l_theta = 0
if help_with_bounds == True:
    max_cliq, best_k = get_lb_clique(graph, 60)
    ub_pre, feas_coloring = get_ub_DSATUR(graph, 100000)
    lb_pre = best_k
    print(f"Upper Bound: {ub_pre}")
    print(f"Lower Bound: {lb_pre}")
    print(f"best clique size found is {best_k}")
    print(f"Lovasz-theta on complement graph : {l_theta}")
    ub = ub_pre
    lb = max(lb_pre, math.ceil(l_theta))
else:
    ub = len(graph.nodes)
    lb = 1
    
print(f"best upper bound: {ub}")    
print(f"best lower bound: {lb}")    

best upper bound: 1000
best lower bound: 1


In [154]:
# len(nx.maximal_independent_set(graph))

In [155]:
# Regenerating maximal cliques
maximal_cliques = nx.find_cliques(graph)

In [156]:
# Pre-processing 1
if PRE_PROCESS == True:
    t1 = time.time()
    cliq = set(max_cliq)

    v_w_pair = [] # v gets the same color as w

    round_num = 0
    while True:
        vnk = set(graph.nodes) - cliq
        vnk_deg = dict(graph.degree(vnk))
        v_notin_k = set(sorted(vnk, key=lambda x: vnk_deg[x]))
        round_num += 1
        removed = 0
        for v in v_notin_k:
            #if v == 246:
            #print(v)
            v_neigh = set(graph.neighbors(v))
            for w in cliq:
                w_neigh = set(graph.neighbors(w))
                if w not in v_neigh and v_neigh.issubset(w_neigh):
                    if graph.has_node(v):
                        graph.remove_node(v)
                        removed += 1
                        # print(f'removing {v}')
                    v_w_pair.append((v,w))
        print(f"removed in round {round_num}: {removed} nodes")
        if removed == 0:
            break
    print(f"Identified {len(set([v for v,w in v_w_pair]))} nodes for removing in Pre-proc 1")
    print(f"{time.time()-t1} seconds for pre-proc 1")

In [157]:
# Preprocessing 2
# Recursive node deletion until max_degree > n_cli - 2
if PRE_PROCESS == True:
    nodes_removed = []
    V = [v for v, deg in list(graph.degree) if deg < best_k - 1]
    iopp = 0
    while V:
        #print(len(V))
        for v in V:
            if graph.degree(v) < best_k - 1:
                iopp += 1
                graph.remove_node(v)
                nodes_removed.append(v)
                V = [v for v, deg in list(graph.degree) if deg < best_k - 1]
                # print(f"removed {iopp} nodes")
                break
    print(f"Nodes removed in pre-proc 2: {len(nodes_removed)}")

In [158]:
if PRE_PROCESS == True:
    print(f"# nodes after pre-processing: {len(graph.nodes)}")
    print(f"# edges after pre-processing: {len(graph.edges)}")

In [159]:
if PRE_PROCESS == True:
    max_cliq_post, lb_post = get_lb_clique(graph, 60)
    ub_post, feas_coloring = get_ub_DSATUR(graph, 100000)

    print("Before Pre-processing")
    print(f"Upper Bound: {ub_pre}")
    print(f"Lower Bound: {lb_pre}")

    print("After Pre-processing")
    print(f"Upper Bound: {ub_post}")
    print(f"Lower Bound: {lb_post}")

    assert ub_post >= ub_pre
    assert lb_post <= lb_pre
    
    #ub = ub_post
    #lb = lb_pre
    
    if ub_post > ub_pre:
        print("UB bad behaviour, resetting")
        ub = ub_pre
    if lb_post < lb_pre:
        print("LB bad behaviour, resetting")
        lb = lb_pre

In [160]:
if PRE_PROCESS == True:
    node_mapping = dict(zip(list(graph.nodes), range(1, len(graph.nodes)+1)))
    reverse_node_mapping = dict(zip(range(1, len(graph.nodes)+1), list(graph.nodes)))
    nx.relabel_nodes(graph, node_mapping, copy=False) # In-place re-labelling

In [161]:
print(f"Upper bound on chromatic number: {ub}")    

Upper bound on chromatic number: 1000


In [162]:
print(lb, ub)

1 1000


In [163]:
assert ub >= lb, "DSATUR Upper bound is lesser than found clique size"

In [165]:
gp_model = gp.Model('GCP_gp') #gp.read('out.lp')

In [167]:
gp_model.setParam('TimeLimit', 600)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [169]:
var_type = (GRB.BINARY if isBIP == True else GRB.CONTINUOUS)
var_type

'B'

In [170]:
# Adding variables
# Colors

w = gp_model.addVars(ub, vtype=var_type, lb=0, ub=1, name='w')
# Vertex-color assignment
x = gp_model.addVars(len(graph.nodes), ub, vtype=var_type, lb=0, ub=1, name='x')

In [171]:
# Warm start MIP with feasible coloring from DSATUR
pairs_start = []
if mip_warm_start == True:
    print("warm start MIP with feas coloring")
    if PRE_PROCESS == True:
        for key in feas_coloring:
            j = feas_coloring[key]
            #print(key, j)
            x[node_mapping[key]-1, j].start = 1.0
            w[j].start = 1.0
            pairs_start.append((node_mapping[key]-1, j))
    else:
        print(' no preprocess')
        for key in feas_coloring:
            j = feas_coloring[key]
            x[key-1, j].start = 1.0
            w[j].start = 1.0
            pairs_start.append((key-1, j))

In [172]:
# Setting lower bound on number of colors
print(l_theta)
if use_lovasz_theta == True and l_theta > 0: 
    gp_model.addConstr(gp.quicksum(w[i] for i in range(len(w))) >= math.ceil(l_theta))

8.307


In [173]:
gp_model.update()
len(gp_model.getVars())


1001000

In [174]:
# Constraints
# if go_easy add Neighbourhood inequalities, otherwise add Adjacency constraints
if go_easy:
    print("going easy")
    for v in graph.nodes:
        N_v = list(graph.neighbors(v))
        # assert len(N_v) == graph.degree(v)
        r = graph.degree(v)
        gp_model.addConstrs((gp.quicksum(x[k-1, j0] for k in N_v) + r*x[v-1, j0] <= r*w[j0] for j0 in range(ub)), name='neigh')
else:
    print("Doing it the hard way.")
    gp_model.addConstrs((x[i-1,j] + x[k-1,j] <= w[j] for i,k in graph.edges for j in range(ub)), name='adj')

going easy


In [175]:
gp_model.update()
len(gp_model.getConstrs())

1000000

In [176]:
# Vertex-Color assignment constraints
#for i in range(len(graph.nodes)):
    #m += mip.xsum(x[i,j] for j in range(len(graph.nodes))) == 1, f"vertex[{i+1}]"
    
gp_model.addConstrs(((gp.quicksum(x[i,j] for j in range(ub))) == 1 for i in range(len(graph.nodes))), name='Vert-col')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [177]:
gp_model.update()
len(gp_model.getConstrs())

1001000

In [178]:
# Removing symmetry
# Activation of color
# for j in range(len(graph.nodes)):
#     m += mip.xsum(x[i][j] for i in range(len(graph.nodes))) >= w[j], f"col[{j+1}]-act"
    
gp_model.addConstrs(((gp.quicksum(x[i,j] for i in range(len(graph.nodes)))) >= w[j] for j in range(ub)), name='Act')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [179]:
gp_model.update()
len(gp_model.getConstrs())

1002000

In [180]:
# use new color, only if previous color is used
gp_model.addConstrs((w[j] >= w[j+1] for j in range(ub-1)), name="newCol")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [181]:
gp_model.update()
len(gp_model.getConstrs())

1002999

In [182]:
#m.objective = mip.minimize(mip.xsum(w[j] for j in range(len(graph.nodes))))
gp_model.setObjective(gp.quicksum(w[i] for i in range(ub)), GRB.MINIMIZE)

In [183]:
gp_model.update()
len(gp_model.getConstrs())

1002999

In [184]:
# fix variables
#precolor_mapping = {}
if variable_fixing == True:
    print("Fixing variables")
    for idx, vertex in enumerate(max_cliq):
        #precolor_mapping[vertex] = idx
        gp_model.setAttr("LB", x[vertex-1,feas_coloring[vertex]], 1)
        gp_model.setAttr("LB", w[feas_coloring[vertex]], 1)


In [185]:
gp_model.update()
len(gp_model.getConstrs())

1002999

In [186]:
gp_model.update()
#gp_model.write('gp_out.lp')

In [187]:
def HeuEdgeCover(G: nx.graph, x_frac: dict):
    # Data: Graph G = (V, E), fractional point x_frac
    # Parameter: Violation tolerance epsilon
    # Result: A collection C = C_1,.......C_k of violated cliques
    
    # x_frac is dict {vertex_name : vertex_relaxation_sol}
    # F - list of vertices where solution is fractional
    # L - list of vertices where solution is LB (0)
    # U - list of vertices where solution is UB (1)
    
    #assert len(x_frac) == len(G.nodes), "Fractional solution not as expected"
    
    # prep
    adj_out_list = [list(graph.neighbors(i)) for i in graph.nodes]
    adj_2d, lens = make_2D_array(adj_out_list)
    # prep ends
    
    INF_MIN = -1e6
    epsilon = .1
    # Counter variables
    clique_counter = 0
    
    violated_cliques = []
    
    F = set(vertex for vertex, sol in x_frac.items() if 0 < sol < 1)
    L = set(vertex for vertex, sol in x_frac.items() if sol <= 0)
    U = set(vertex for vertex, sol in x_frac.items() if sol >= 1)
    
    
    # print(L, F, U)
    
    G_prime = nx.subgraph(G, F).copy()
    
    V_prime = list(G_prime.nodes)
    
    # print(G_prime.edges)
    
    f_FL = {node: deg*x_frac[node] for node, deg in G.degree(F.union(L))}
    f_U = {node: INF_MIN for node in U}
    f_ = {**f_FL, **f_U} # merge two dicts into one
    
    f = np.asarray([val for key, val in f_.items()])
    
    #if len(f) != len(G.nodes):
    #    missing_set = set(range(1, num_v+1)) - set(f.keys())
    #    print(missing_set)
    #    for sss in missing_set:
    #        print(x_frac[sss])
        #print(len(f), f.keys())
    #assert len(f) == len(G.nodes)
    
    V_prime.sort(reverse=False, key=lambda x: f[x-1])
    
    for i in V_prime:
        S = set(j for j in G_prime.neighbors(i) if f[j-1] > 0)
        UB_viol = x_frac[i] + sum(x_frac[j] for j in G.neighbors(i)) - 1
        while UB_viol > epsilon and S:
            j = max((k for k in S), key=lambda x: f[x-1]) #argmax(f[k])
            # print('HEC', [i,j])
            C = GreedyGrow(adj_2d, lens, f, np.asarray([i,j]))
            S = S.difference(C)
            G_prime.remove_edges_from(list(nx.subgraph(G_prime, C).edges))
            UB_viol = UB_viol - sum(x_frac[k] for k in set(C).difference({i}))
            if sum(x_frac[k] for k in C) > 1 + epsilon:
                clique_counter += 1
                violated_cliques.append(C)
    return violated_cliques

In [188]:
def make_2D_array(lis):
    """Funciton to get 2D array from a list of lists
    """
    n = len(lis)
    lengths = np.array([len(x) for x in lis])
    max_len = max(lengths)
    arr = np.zeros((n, max_len), dtype=np.int32)

    for i in range(n):
        arr[i, :lengths[i]] = lis[i]
    return arr, lengths

In [189]:
@njit
def GreedyGrow(G, lens, weights: list, C: list):
    # Data: Graph G = (V, E); weights f_j, j in V; clique C in G
    # Result: A Clique C_bar in G including C
    # f is a dict of {node_name : weights}
    
    # G is a 2D adjList
    
    #if len(f) != num_v:
    #    print("wor", len(f), f.keys())
    node_list = np.asarray(list(range(1, len(G)+1)), dtype=np.int32)
    G_nodes = set(node_list) # index starting from 1 to |V|
    
    def neighbors(G, lens, node):
        idx = node-1
        return G[idx][:lens[idx]]
    
    def non_neighbors(G, lens, node, G_nodes):
        idx = node-1
        neigh = set(G[idx][:lens[idx]])
        return G_nodes.difference(neigh)
    
    # equivalent func for dict of node: weights, now we'll access an np.array (idx changes to 0 to n-1)
    def f(weights, node):
        return weights[node-1]
    
    # Compute common neighbours of all nodes in a list
    first_node = C[0]
    
    f_set = set(neighbors(G, lens, first_node))
    
    #print('GG', C)
    for node in C[1:]:
        c_n = set(neighbors(G, lens, node))
        f_set = f_set.intersection(c_n)

    #print('GG', f_set)
        
    # G_bar = nx.subgraph(G, f_set)
    #V_bar = set(G_bar.nodes)
    
    V_bar = f_set

    # assert V_bar == f_set, "Wait, what??"
    C_bar = list(C) # copy C to initialise C_bar
    
    while V_bar:    # while V_bar is not empty
        #print('GG', V_bar, f)
        val_max = -1
        arg_max = -1
        for k in V_bar:
            if weights[k-1] > val_max:
                arg_max = k
        #j = max((k for k in V_bar), key=lambda x: f[x])  # argmax(f(k) for k in V_bar)
        j = arg_max
        #print('GG', j)
        C_bar.append(j)
        non_neigh = non_neighbors(G, lens, j, G_nodes) # original code

        V_bar = V_bar.difference(non_neigh.union({j}))
        
    return C_bar
    

In [191]:
gp_model.params.Heuristics =  0.05
gp_model.params.NoRelHeuristic = -1 #0
gp_model.params.PreCrush = 1
gp_model.params.Cuts = -1
gp_model.params.method = -1 # 5 - both primal & dual concurrent, -1 - default (concurrent all)
gp_model.params.logFile = SOLVER_LOG_PATH + "\\" + instance_name + ".log"

Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Parameter NoRelHeuristic unchanged
   Value: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Parameter Cuts unchanged
   Value: -1  Min: -1  Max: 3  Default: -1
Parameter method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter logFile to F:\Research\Problem sets\Coloring\CP\SolverLog\DSJC1000.1.log
   Prev:   Default: 


In [192]:
graph_nodes = list(graph.nodes)
add_user_cuts = False 

#pr = cProfile.Profile()

In [ ]:
def mycallback(model, where):
    if where == GRB.Callback.MIPNODE:
        # print(model.cbGetNodeRel(model._vars))
        #all_cli = nx.find_cliques(graph)
        #all_cli_set = set(frozenset(i) for i in all_cli)
        
        #validate_hec_cliques = True
        
        status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
        if status == GRB.OPTIMAL:
            rel_sol = model.cbGetNodeRel(model._vars)
            #get_sol_val = lambda var: var.x
            

            xx = np.array(rel_sol[ub:])
            assert len(xx) == num_v*ub
            #print(xx)
            #v_col_rel_sol = np.array(list(map(get_sol_val, xx))).reshape(num_v,ub)
            
            v_col_rel_sol = xx.reshape(num_v,ub)
            
            # my_f = list(dict(zip(list(graph.nodes), v_col_rel_sol[:,j0])) for j0 in range(ub))
            my_f = list(dict(zip(graph_nodes, v_col_rel_sol[:,j0])) for j0 in range(ub))

            #sep_start_t = time.time()

            v_cliques = [HeuEdgeCover(graph, x_f) for x_f in my_f]

            #sep_elapsed_time += (time.time() - sep_start_t)

            num_cuts_found = sum(len(v_clique) for v_clique in v_cliques)
            #print("Number of Violated Cliques found by HEC: ", num_cuts_found)
            
            cut_num=0
            
            for j0 in range(ub):
                for each_clq in v_cliques[j0]:
                    cut_num += 1
                    if cut_num > 1000:
                        break
                    model.cbCut(gp.quicksum(x[i-1, j0] for i in each_clq) <= w[j0])
                if cut_num > 1000:
                    break
                    
            
            #if rel[0] + rel[1] > 1.1:
            #    model.cbCut(model._vars[0] + model._vars[1] <= 1)


        
        #total_cuts += cut_num

#gp_model.params.Cuts = 0
#gp_model.params.Heuristics = 0
#gp_model.params.PreCrush = 1
#gp_model.params.logFile = SOLVER_LOG_PATH + "\\" + instance_name + ".log"

In [193]:
gp_model.reset()
gp_model._vars = gp_model.getVars()
def model_solve():
    if add_user_cuts == True:
        gp_model.optimize(mycallback)
    else:
        gp_model.optimize()

#pr.enable()
model_solve()
#pr.disable()

Discarded solution information
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1002999 rows, 1001000 columns and 103260998 nonzeros
Model fingerprint: 0x189d4eb5
Variable types: 0 continuous, 1001000 integer (1001000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 0 columns (presolve time = 7s) ...
Presolve removed 0 rows and 0 columns (presolve time = 17s) ...
Presolve time: 35.98s

Explored 0 nodes (0 simplex iterations) in 50.99 seconds
Thread count was 1 (of 12 available processors)

Solution count 0

Best objective -, best bound -, gap -


GurobiError: Out of memory

In [161]:
# pr.print_stats(sort='time')

         185561567 function calls (184734682 primitive calls) in 136.443 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   45.314   45.314  136.443  136.443 <ipython-input-160-9dc069662d48>:3(model_solve)
   137040    7.517    0.000   84.737    0.001 <ipython-input-148-9dd43b677883>:37(HeuEdgeCover)
   137040    6.245    0.000   11.441    0.000 <ipython-input-148-9dd43b677883>:69(<dictcomp>)
 15162429    4.504    0.000    4.504    0.000 <ipython-input-148-9dd43b677883>:57(<genexpr>)
 15559904    4.464    0.000    5.196    0.000 reportviews.py:442(__iter__)
   534515    4.141    0.000    4.141    0.000 <ipython-input-148-9dd43b677883>:56(<genexpr>)
  9614901    3.836    0.000    5.279    0.000 coreviews.py:322(new_node_ok)
   336736    3.591    0.000    3.591    0.000 <ipython-input-148-9dd43b677883>:58(<genexpr>)
    50248    3.291    0.000   91.089    0.002 <ipython-input-137-38c2743e7303>:1(mycallback)
  4943723

In [144]:
def lp_cut_plane():
    lp_start_t = time.time()

    iter_num = 0
    next_iter = True
    node_cnt = 0
    sep_elapsed_time = 0
    total_cuts = 0
    cut_num = 0

    while next_iter and iter_num < 100 and time.time() - lp_start_t < 3600:
        iter_num += 1
        cut_num = 0
        print(f"Iteration {iter_num} begins:\n")
        gp_relax_model.optimize()
        rel_sol = gp_relax_model.getVars()

        int_inf = len([sol for sol in rel_sol if 0 < sol.x < 1])
        if int_inf == 0:
            print(f"Integer Optimal solution found at cutting plane round {iter_num}")
            break

        obj_val = gp_relax_model.objVal
        #v_col_var = np.array(gp_relax_model._vars[ub:]).reshape(num_v,ub)
        get_sol_val = lambda var: var.x

        xx = np.array(rel_sol[ub:])

        v_col_rel_sol = np.array(list(map(get_sol_val, xx))).reshape(num_v,ub)

        my_f = list(dict(zip(list(graph.nodes), v_col_rel_sol[:,j0])) for j0 in range(ub))

        sep_start_t = time.time()

        v_cliques = [HeuEdgeCover(graph, x_f) for x_f in my_f]

        sep_elapsed_time += (time.time() - sep_start_t)

        num_cuts_found = sum(len(v_clique) for v_clique in v_cliques)
        print("Number of Violated Cliques found by HEC: ", num_cuts_found)

        if num_cuts_found == 0:
            next_iter = False
            print(f'Terminating at cutting plane round {iter_num}')
        else:
            total_cuts += cut_num

            for j0 in range(ub):
                for each_clq in v_cliques[j0]:
                    cut_num += 1
                    if cut_num > 1000:
                        break
                if cut_num > 1000:
                    break
                gp_relax_model.addConstr(gp.quicksum(x[i-1, j0] for i in each_clq) <= w[j0], name=f"UserClq[{cut_num}]_node[{node_cnt}]")
            gp_relax_model.optimize()
        node_cnt += 1

    lp_end_t = time.time()
    print()
    print()
    print(f"Iteration end: {iter_num}")
    print(f"Cutting plane round + clique separations took {lp_end_t - lp_start_t} seconds.")
    print(f"Clique separations took {sep_elapsed_time} seconds.")
    print(f"Total Clique found {total_cuts}")

In [ ]:
p_end_t = time.time()

print(f"The whole program took {p_end_t - p_start_t} seconds.")